###Import Library###

In [1]:
import nltk

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import json

###Load the json data###

In [5]:
with open('pavbhaji.json', 'r',encoding='utf-8') as file:
    data = json.load(file)


In [25]:
data[0]

{'dimensions': {'height': 734, 'width': 640},
 'display_url': 'https://instagram.fpnq3-1.fna.fbcdn.net/vp/352198856dbdacba4b84a4a9a77ad087/5B8644A9/t51.2885-15/e15/37599530_291077411445016_3382263118984904704_n.jpg',
 'edge_liked_by': {'count': 3797},
 'edge_media_preview_like': {'count': 3797},
 'edge_media_to_caption': {'edges': [{'node': {'text': 'TAG A PAV BHAJI FANATIC 😋😍 PC @exploringplates For more food updates follow @ekplate To get featured tag us on your pictures ❤ .\n.\n#ekplate #nomnom #nom #delhi_igers #eeeeeats #foodaholic #foodblogger #foodphotography #foodcoma #foodgram #things2doinmumbai #foodblog #foodgasm #foodlove #india  #foodie #yummy  #bombay #mumbai #delhi #pune  #bangalore #streetfood  #ekplatepavbhaji #butter #vadapav #pavbhaji #mumbaifood'}}]},
 'edge_media_to_comment': {'count': 52},
 'id': '1834712933156555738',
 'is_video': True,
 'location': None,
 'owner': {'id': '1919686029'},
 'shortcode': 'Bl2NbElnIva',
 'tags': ['vadapav',
  'foodgram',
  'foodphotog

In [7]:
import json

def load_json_data(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
    return data

In [8]:
data = load_json_data("pavbhaji.json")

#### Extract texts and labels###


In [9]:
texts = []
labels = []
for entry in data:
    try:
        text = entry["edge_media_to_caption"]["edges"][0]["node"]["text"]
        label = 1 if "#pavbhaji" in text.lower() else 0
        texts.append(text)
        labels.append(label)

        #print(f"Text: {text}")
        #print(f"Label: {label}")

    except (KeyError, IndexError) as e:
        print(f"Skipping entry due to error: {e}")
        print(entry)

Skipping entry due to error: list index out of range
{'comments_disabled': False, 'dimensions': {'height': 1080, 'width': 1080}, 'display_url': 'https://instagram.fpnq3-1.fna.fbcdn.net/vp/c6dac55b57c2e87ecb96315519b1459c/5C02F331/t51.2885-15/e35/38885028_254267458551198_8280479196141060096_n.jpg', 'edge_liked_by': {'count': 17}, 'edge_media_preview_like': {'count': 17}, 'edge_media_to_caption': {'edges': []}, 'edge_media_to_comment': {'count': 2}, 'id': '1850713497443087014', 'is_video': False, 'location': {'has_public_page': True, 'id': '216978098', 'name': 'Mumbai, Maharashtra', 'slug': 'mumbai-maharashtra'}, 'owner': {'id': '8430771130'}, 'shortcode': 'BmvDh7UHbKm', 'taken_at_timestamp': 1534842262, 'thumbnail_resources': [{'config_height': 150, 'config_width': 150, 'src': 'https://instagram.fpnq3-1.fna.fbcdn.net/vp/84b28b95442e3eedaa87eed3dde75f26/5C314973/t51.2885-15/e35/s150x150/38885028_254267458551198_8280479196141060096_n.jpg'}, {'config_height': 240, 'config_width': 240, 'src

###Text Preprocessing###

In [10]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z]', ' ', text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

cleaned_text_features = [preprocess_text(text) for text in texts]

In [26]:
cleaned_text_features[1:5]

['desi style paav bhaji good afternoon instafamily ingredients potatoes cauliflower carrot cup peas cups water tbsp oil tbsp ginger garlic paste onions finely chopped tomatoes finaly chopped tbsp red chilli powder tbsp kashmiri red chilli tbsp pav bhaji masala salt taste coriander garnish recipe pressure cook roughly chopped potatoes cauliflower carrot peas cups water whistles mash boiled veggies keep aside heat oil wok add ginger garlic paste cook till aroma fades away add onions cook till turn light brown add tomatoes cook till corner leaves oil add red chilli powder kashmiri chilli powder pav bhaji masala mix add mashed veggies salt mix cook mints adjust consistency adding water stir mash veggies garnish coriander serve hot pav follow sofaridabad homemade food updates please turn post notification use sofaridabad get featured page pavbhaji streetstyle homemadewithlove homecooked cooking streetfood mumbaifoodie views eat tastyfood favourite dish carrot vegetarian vegetables healthyli

###Train Test split###

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(cleaned_text_features, labels, test_size=0.2, random_state=42)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

###TF IDF###

In [15]:
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

###Import and Apply Model###

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

In [17]:
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

MultinomialNB()

###Predict the Model###

In [18]:
predictions = model.predict(X_test_vectorized)

###Evaluate the Model###

In [19]:
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, predictions))

Accuracy: 0.9833333333333333
              precision    recall  f1-score   support

           0       1.00      0.17      0.29         6
           1       0.98      1.00      0.99       294

    accuracy                           0.98       300
   macro avg       0.99      0.58      0.64       300
weighted avg       0.98      0.98      0.98       300



In [21]:
X_test[50]

'say pavbhaji fastfood fastfoodporn foodie foody ymmy lajpatnagar delhigram sundayfunday northindianfood bloggerstyle'

In [22]:
y_test[50]

1

In [23]:
confusion_matrix(y_test, predictions)

array([[  1,   5],
       [  0, 294]])